In [0]:
fileName = "dbfs:/FileStore/tables/people/dim-1.csv"

In [0]:
df_people=spark.read.format("csv").option("header",True).option("inferSchema", True).load(fileName)

In [0]:
df_people.printSchema()

root
-- h: integer (nullable = true)
-- w: integer (nullable = true)
-- t: integer (nullable = true)

In [0]:
df_people.count()

Out[111]: 32

In [0]:
df_people.describe().show()

+-------+------------------+------------------+------------------+
summary| h| w| t|
+-------+------------------+------------------+------------------+
 count| 32| 32| 32|
 mean| 174.125| 68.09375| 0.46875|
 stddev|12.610159754430477|14.785257205918377|0.5070073486774164|
 min| 155| 45| 0|
 max| 201| 101| 1|
+-------+------------------+------------------+------------------+

In [0]:
col=["h","w"]


In [0]:
df_people.groupby("t").count().show()

+---+-----+
 t|count|
+---+-----+
 1| 15|
 0| 17|
+---+-----+

In [0]:
from pyspark.ml.feature import VectorAssembler
vectorizer = VectorAssembler()
vectorizer.setInputCols(col)
vectorizer.setOutputCol("features")

Out[115]: VectorAssembler_5df3a6a157b6

In [0]:
seed = 180009193
(split20DF, split80DF) = df_people.randomSplit([0.2,0.8],seed)

# Let's cache these datasets for performance
testSetDF = split20DF.cache()
trainingSetDF = split80DF.cache()

In [0]:
df_people.show(40)

+---+---+---+
 h| w| t|
+---+---+---+
167| 60| 1|
155| 51| 1|
178| 70| 0|
159| 50| 1|
198| 92| 0|
181| 90| 0|
164| 49| 1|
177| 71| 0|
181| 70| 0|
189| 79| 0|
177| 64| 1|
167| 56| 1|
168| 60| 1|
190|101| 0|
201| 89| 0|
171| 69| 1|
170| 87| 0|
161| 55| 1|
180| 73| 0|
178| 75| 0|
178| 69| 0|
156| 47| 1|
170| 56| 1|
156| 54| 1|
178| 66| 0|
173| 80| 0|
155| 45| 1|
198| 88| 0|
188| 77| 0|
170| 55| 1|
164| 60| 1|
174| 71| 0|
+---+---+---+

In [0]:
testSetDF.show()

+---+---+---+
 h| w| t|
+---+---+---+
164| 60| 1|
168| 60| 1|
170| 55| 1|
171| 69| 1|
174| 71| 0|
177| 71| 0|
178| 69| 0|
181| 70| 0|
188| 77| 0|
189| 79| 0|
198| 88| 0|
+---+---+---+

In [0]:
trainingSetDF.show()

+---+---+---+
 h| w| t|
+---+---+---+
155| 45| 1|
155| 51| 1|
156| 47| 1|
156| 54| 1|
159| 50| 1|
161| 55| 1|
164| 49| 1|
167| 56| 1|
167| 60| 1|
170| 56| 1|
170| 87| 0|
173| 80| 0|
177| 64| 1|
178| 66| 0|
178| 70| 0|
178| 75| 0|
180| 73| 0|
181| 90| 0|
190|101| 0|
198| 92| 0|
+---+---+---+
only showing top 20 rows

In [0]:
from pyspark.ml.classification import LogisticRegression
# ************ RANDOM FOREST ********************
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

lr = LogisticRegression()

print(lr.explainParams())


aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
upperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
upperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

https://commons.wikimedia.org/wiki/File:Logistic-curve.svg

In [0]:
from pyspark.ml.regression import *
from pyspark.ml import Pipeline

lr = LogisticRegression(featuresCol="features", labelCol="t", predictionCol="Predicted_t", maxIter=100, regParam=0.3, elasticNetParam=0.8)

pipeline = Pipeline(stages=[vectorizer, lr])                      
  
lrmodel  = pipeline.fit(trainingSetDF)

In [0]:
print(lrmodel.stages[-1].coefficients)  
print(lrmodel.stages[-1].intercept)  

[-0.0014843753791806678,-0.03675452303919101]
2.8222624619620538

In [0]:
predictions = lrmodel.transform(testSetDF)


In [0]:
predictions.select("h","w","t","Predicted_t").show()

+---+---+---+-----------+
 h| w| t|Predicted_t|
+---+---+---+-----------+
164| 60| 1| 1.0|
168| 60| 1| 1.0|
170| 55| 1| 1.0|
171| 69| 1| 1.0|
174| 71| 0| 0.0|
177| 71| 0| 0.0|
178| 69| 0| 1.0|
181| 70| 0| 0.0|
188| 77| 0| 0.0|
189| 79| 0| 0.0|
198| 88| 0| 0.0|
+---+---+---+-----------+